### 정수 인코딩(Integer Encoding)

- 자연어 처리는 텍스트 데이터를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 만드는 것이 핵심.
- 정수 인코딩을 수행하여 텍스트 데이터에 고유한 인덱스를 부여.
- 이러한 인코딩 과정은 전처리 과정에서 필수적이며 각 단어의 등장 빈도에 따라 인덱스를 부여하는 것이 일반적.
- 단어 수를 5,000개로 제한하는 것은 모델 학습에 필요한 메모리와 계산 자원(리소스, 메모리 등)을 줄이기 위함.  
    => 등장 빈도가 낮은 단어는 제외하고 상위 5,000개 단어만 선택하는 것이 일반적.

In [31]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

### 인코딩
#### 토큰화 + 정제/정규화

In [32]:

######### 토큰화 과정 #########
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# 문장 토큰화
# sent_tokenize() : 문장 단위로 토큰화
sentences = sent_tokenize(raw_text)

# 영어 불용어 리스트
stop_words = set(stopwords.words('english'))

# 단어 사전(key=단어, value=빈도수)
vocab = {}

# 토큰화/정제/정규화 처리 결과
preprocessed_sentences = []

# 토큰 만큼 반복
for sent in sentences:
    sent = sent.lower()  # 대소문자 정규화(소문자 변환)
    words = word_tokenize(sent)  # 단어 토큰화
    # 불용어 제거
    words = [word for word in words if word not in stop_words] 
    # 단어 길이가 2 이하면 제거 (필터링)
    filtered_tokens = [word for word in words if len(word) > 2]
    
    # vocab 사전에 단어가 없으면 추가, 있으면 빈도수 증가
    for word in filtered_tokens:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
    preprocessed_sentences.append(filtered_tokens)
    


In [33]:
# 공통 모듈에서 함수 임포트
import sys
import os

# 상위 디렉토리의 utils 모듈을 임포트하기 위한 경로 추가
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))

# 또는 더 간단하게:
# sys.path.append('../utils')

from text_preprocessing import (
    preprocess_text_for_encoding,
    create_word_to_index_mapping,
    encode_sentences
)

In [34]:
######### utils 모듈 토큰화 함수를 사용하여 전처리 수행 #########
vocab, preprocessed_sentences = preprocess_text_for_encoding(raw_text)

# 결과 확인
print("📊 단어 사전 크기:", len(vocab))
print("\n🔤 상위 10개 빈도수 단어:")
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
for word, freq in sorted_vocab[:10]:
    print(f"  {word}: {freq}회")

print(f"\n📄 전처리된 문장 수: {len(preprocessed_sentences)}")
print(f"📝 첫 번째 문장 예시: {preprocessed_sentences[0][:10]}...")

📊 단어 사전 크기: 107

🔤 상위 10개 빈도수 단어:
  prince: 9회
  little: 6회
  pilot: 4회
  rose: 3회
  fox: 3회
  young: 2회
  planet: 2회
  earth: 2회
  story: 2회
  plane: 2회

📄 전처리된 문장 수: 13
📝 첫 번째 문장 예시: ['little', 'prince', 'written', 'antoine', 'poetic', 'tale', 'young', 'prince', 'travels', 'home']...


### 🔧 함수화의 장점

**1. 재사용성 (Reusability)**
- 다른 텍스트 데이터에도 동일한 전처리 적용 가능
- 매번 같은 코드를 작성할 필요 없음

**2. 매개변수 조정 (Flexibility)**
- `min_word_length`: 최소 단어 길이 조정
- `language`: 다른 언어의 불용어 사용 가능

**3. 코드 가독성 (Readability)**
- 복잡한 전처리 로직이 함수 안에 캡슐화
- 메인 코드가 더 깔끔하고 이해하기 쉬움

**4. 유지보수 (Maintainability)**
- 전처리 로직 수정 시 한 곳만 변경하면 됨
- 버그 수정이나 기능 개선이 용이

In [35]:
# 모듈화된 함수로 완전한 정수 인코딩 수행
print("🔄 모듈화된 함수로 전처리 시작...")

# 1단계: 전처리
vocab, preprocessed_sentences = preprocess_text_for_encoding(raw_text)

# 2단계: 단어-인덱스 매핑 생성
word_to_index, index_to_word = create_word_to_index_mapping(vocab, max_vocab_size=100)

# 3단계: 문장들을 정수로 인코딩
encoded_sentences = encode_sentences(preprocessed_sentences, word_to_index)

# 결과 출력
print(f"✅ 어휘 사전 크기: {len(word_to_index)}")
print(f"✅ 원본 문장 수: {len(encoded_sentences)}")
print(f"\n📝 첫 번째 문장 예시:")
print(f"   원본: {preprocessed_sentences[0][:5]}...")
print(f"   인코딩: {encoded_sentences[0][:5]}...")

🔄 모듈화된 함수로 전처리 시작...
✅ 어휘 사전 크기: 102
✅ 원본 문장 수: 13

📝 첫 번째 문장 예시:
   원본: ['little', 'prince', 'written', 'antoine', 'poetic']...
   인코딩: [3, 2, 17, 18, 19]...


### 🚀 다른 노트북에서 사용하는 방법

**1. 같은 디렉토리의 노트북 (예: `02_preprocessing/` 내)**
```python
import sys
sys.path.append('../utils')
from text_preprocessing import preprocess_text_for_encoding
```

**2. 다른 디렉토리의 노트북 (예: `01_text_analysis/` 내)**
```python
import sys
sys.path.append('../utils')
from text_preprocessing import preprocess_text_for_encoding
```

**3. 전체 패키지 임포트**
```python
import sys
sys.path.append('../utils')
import text_preprocessing as tp

# 사용 예시
vocab, sentences = tp.preprocess_text_for_encoding(text)
```
 

#### 빈도수 기반 정제

In [36]:
print(f'vocab: {vocab}')

# 빈도수 기반 역순 정렬
# sorted() => 내장 정렬 함수
# vocab.items() => (단어, 빈도수) 튜플 리스트
# key=lambda x: x[1] 
#     => 빈도수 기준 정렬 
#     => lambda는 익명 함수 정의
#     => x: x[1] => 각 튜플 x에 대해 두 번째 요소(빈도수)를 반환
# reverse=True => 내림차순 정렬
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
print(f'sorted_vocab: {sorted_vocab}') 
# 결과 : key: 단어, value: 빈도수 튜플 리스트 (빈도수 내림차순)

vocab: {'little': 6, 'prince': 9, 'written': 1, 'antoine': 1, 'poetic': 1, 'tale': 1, 'young': 2, 'travels': 1, 'home': 1, 'planet': 2, 'earth': 2, 'story': 2, 'begins': 1, 'pilot': 4, 'stranded': 1, 'sahara': 1, 'desert': 1, 'plane': 2, 'crashes': 1, 'trying': 1, 'fix': 1, 'meets': 2, 'mysterious': 1, 'boy': 1, 'comes': 1, 'small': 1, 'asteroid': 2, 'called': 1, 'lives': 1, 'alone': 1, 'rose': 3, 'loves': 1, 'deeply': 1, 'recounts': 1, 'journey': 1, 'describing': 1, 'visits': 1, 'several': 1, 'planets': 1, 'inhabited': 1, 'different': 1, 'character': 1, 'king': 1, 'vain': 1, 'man': 1, 'drunkard': 1, 'businessman': 1, 'geographer': 1, 'fox': 3, 'encounters': 1, 'learns': 1, 'valuable': 1, 'lessons': 2, 'love': 2, 'responsibility': 1, 'nature': 1, 'adult': 1, 'behavior': 1, 'various': 1, 'creatures': 1, 'including': 1, 'teaches': 1, 'relationships': 1, 'importance': 2, 'taming': 1, 'means': 1, 'building': 1, 'ties': 1, 'others': 1, 'famous': 1, 'line': 1, 'become': 1, 'responsible': 1, 

In [37]:
# 인덱스 단어 사전 생성1
# 튜플의 리스트 형태로 된 sorted_vocab에서 단어만 추출
# enumerate() => 인덱스와 값을 동시에 추출
word_to_index = {word: index + 1 for index, (word, _) in enumerate(sorted_vocab)}
print(f'word_to_index: {word_to_index}')
# 결과 : key: 단어, value: 인덱스+1 (0은 패딩 토큰으로 예약)
# ✨ 빈도수가 높은 단어일수록 낮은 인덱스를 가짐
 

word_to_index: {'prince': 1, 'little': 2, 'pilot': 3, 'rose': 4, 'fox': 5, 'young': 6, 'planet': 7, 'earth': 8, 'story': 9, 'plane': 10, 'meets': 11, 'asteroid': 12, 'lessons': 13, 'love': 14, 'importance': 15, 'written': 16, 'antoine': 17, 'poetic': 18, 'tale': 19, 'travels': 20, 'home': 21, 'begins': 22, 'stranded': 23, 'sahara': 24, 'desert': 25, 'crashes': 26, 'trying': 27, 'fix': 28, 'mysterious': 29, 'boy': 30, 'comes': 31, 'small': 32, 'called': 33, 'lives': 34, 'alone': 35, 'loves': 36, 'deeply': 37, 'recounts': 38, 'journey': 39, 'describing': 40, 'visits': 41, 'several': 42, 'planets': 43, 'inhabited': 44, 'different': 45, 'character': 46, 'king': 47, 'vain': 48, 'man': 49, 'drunkard': 50, 'businessman': 51, 'geographer': 52, 'encounters': 53, 'learns': 54, 'valuable': 55, 'responsibility': 56, 'nature': 57, 'adult': 58, 'behavior': 59, 'various': 60, 'creatures': 61, 'including': 62, 'teaches': 63, 'relationships': 64, 'taming': 65, 'means': 66, 'building': 67, 'ties': 68, '

In [38]:
# 인덱스 단어 사전 생성2
index_to_word = {index: word for word, index in word_to_index.items()}
print(f'index_to_word: {index_to_word}')
# 결과 : key: 인덱스, value: 단어 
# ✨ 2번 등장하는 단어는 인덱스 15번까지 이므로, vocab 사전의 크기를 15로 제한

index_to_word: {1: 'prince', 2: 'little', 3: 'pilot', 4: 'rose', 5: 'fox', 6: 'young', 7: 'planet', 8: 'earth', 9: 'story', 10: 'plane', 11: 'meets', 12: 'asteroid', 13: 'lessons', 14: 'love', 15: 'importance', 16: 'written', 17: 'antoine', 18: 'poetic', 19: 'tale', 20: 'travels', 21: 'home', 22: 'begins', 23: 'stranded', 24: 'sahara', 25: 'desert', 26: 'crashes', 27: 'trying', 28: 'fix', 29: 'mysterious', 30: 'boy', 31: 'comes', 32: 'small', 33: 'called', 34: 'lives', 35: 'alone', 36: 'loves', 37: 'deeply', 38: 'recounts', 39: 'journey', 40: 'describing', 41: 'visits', 42: 'several', 43: 'planets', 44: 'inhabited', 45: 'different', 46: 'character', 47: 'king', 48: 'vain', 49: 'man', 50: 'drunkard', 51: 'businessman', 52: 'geographer', 53: 'encounters', 54: 'learns', 55: 'valuable', 56: 'responsibility', 57: 'nature', 58: 'adult', 59: 'behavior', 60: 'various', 61: 'creatures', 62: 'including', 63: 'teaches', 64: 'relationships', 65: 'taming', 66: 'means', 67: 'building', 68: 'ties', 6

In [39]:
vocab_size =15

# 빈도수 기반 정제
# 단어 사전에서 vocab_size 초과하는 단어 제거 

# word_to_index.items() => 기존 word_to_index 딕셔너리에서 (단어, 인덱스) 쌍을 가져옴
# for word, index in ... => 각 (단어, 인덱스) 튜플을 순회하면서 word에는 단어, index에는 인덱스 값이 할당됨
# if index <= vocab_size => 인덱스가 vocab_size 이하인 경우에만 조건을 만족
# {word: index ...} => 조건을 만족하는 (단어, 인덱스) 쌍으로 새로운 딕셔너리를 생성
word_to_idx = {word: index for word, index in word_to_index.items() if index <= vocab_size}

word_to_idx

# 빈도수를 기반으로 라벨링(단어 매핑)함.
# 빈도수가 낮은 단어를 노이즈로 간주해서 제거.

{'prince': 1,
 'little': 2,
 'pilot': 3,
 'rose': 4,
 'fox': 5,
 'young': 6,
 'planet': 7,
 'earth': 8,
 'story': 9,
 'plane': 10,
 'meets': 11,
 'asteroid': 12,
 'lessons': 13,
 'love': 14,
 'importance': 15}

### oov 처리
**OOV(Out Of Vocabulary)** : 단어사전에 정의되어 있지 않은 단어

In [ ]:
word_to_idx['OOV'] = len(word_to_idx) + 1  # OOV 토큰에 새로운 인덱스 할당
print(f'word_to_idx with OOV: {word_to_idx}')
# 결과 : key: 단어, value: 인덱스 + OOV 토큰 추가
# 결과 값 마지막에 'OOV': 16 => OOV 토큰에 대한 인덱스 

word_to_idx with OOV: {'prince': 1, 'little': 2, 'pilot': 3, 'rose': 4, 'fox': 5, 'young': 6, 'planet': 7, 'earth': 8, 'story': 9, 'plane': 10, 'meets': 11, 'asteroid': 12, 'lessons': 13, 'love': 14, 'importance': 15, 'OOV': 16}


#### 수열처리 (=정수 인코딩)

In [ ]:
encoded_sentences = []
oov_idx = word_to_idx['OOV']  # OOV 토큰의 인덱스

# preprocessed_sentences : 전처리된 문장 리스트
for sent in preprocessed_sentences: 
    encoded_sentence = [word_to_idx.get(token, oov_idx) for token in sent]
    print(sent) # 원본 문장 토큰
    print(encoded_sentence) # 인덱스 출력 , 16은 OOV 토큰 인덱스
    print()
    encoded_sentences.append(encoded_sentence)
    
    # 결과 : 자연어 문장이 정수 시퀀스로 변환됨
    # 단어가 word_to_idx 사전에 없으면 OOV 토큰 인덱스로 대체

['little', 'prince', 'written', 'antoine', 'poetic', 'tale', 'young', 'prince', 'travels', 'home', 'planet', 'earth']
[2, 1, 16, 16, 16, 16, 6, 1, 16, 16, 7, 8]

['story', 'begins', 'pilot', 'stranded', 'sahara', 'desert', 'plane', 'crashes']
[9, 16, 3, 16, 16, 16, 10, 16]

['trying', 'fix', 'plane', 'meets', 'mysterious', 'young', 'boy', 'little', 'prince']
[16, 16, 10, 11, 16, 6, 16, 2, 1]

['little', 'prince', 'comes', 'small', 'asteroid', 'called', 'lives', 'alone', 'rose', 'loves', 'deeply']
[2, 1, 16, 16, 12, 16, 16, 16, 4, 16, 16]

['recounts', 'journey', 'pilot', 'describing', 'visits', 'several', 'planets']
[16, 16, 3, 16, 16, 16, 16]

['planet', 'inhabited', 'different', 'character', 'king', 'vain', 'man', 'drunkard', 'businessman', 'geographer', 'fox']
[7, 16, 16, 16, 16, 16, 16, 16, 16, 16, 5]

['encounters', 'prince', 'learns', 'valuable', 'lessons', 'love', 'responsibility', 'nature', 'adult', 'behavior']
[16, 1, 16, 16, 13, 14, 16, 16, 16, 16]

['earth', 'little', 'princ

### kears Tokenizer

- tensorflow : NLP(자연어) 처리를 위한 라이브러리
- 딥러닝 모델 사용을 위한 라이브러리 파이토치와 같음.

In [46]:
!pip install tensorflow

   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   - -------------------------------------- 10.0/332.0 MB 52.5 MB/s eta 0:00:07
   -- ------------------------------------- 23.1/332.0 MB 58.6 MB/s eta 0:00:06
   --- ------------------------------------ 28.0/332.0 MB 45.9 MB/s eta 0:00:07
   ---- ----------------------------------- 40.9/332.0 MB 49.7 MB/s eta 0:00:06
   ------ --------------------------------- 50.6/332.0 MB 49.1 MB/s eta 0:00:06
   ------- -------------------------------- 61.3/332.0 MB 49.4 MB/s eta 0:00:06
   -------- ------------------------------- 71.3/332.0 MB 49.3 MB/s eta 0:00:06
   --------- ------------------------------ 77.9/332.0 MB 46.9 MB/s eta 0:00:06
   ---------- ----------------------------- 85.5/332.0 MB 45.8 MB/s eta 0:00:06
   ----------- ---------------------------- 91.5/332.0 MB 44.1 MB/s eta 0:00:06
   ----------- ---------------------------- 97.5/332.0 MB 43.0 MB/s eta 0:00:06
   ------------ -------------------------- 103.8/

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenizer 객체 생성
#  - num_words : 최대 단어 수 제한 , vocab_size(상위 15개)
#  - oov_token : OOV 토큰 지정 (vocab에 없는 단어를 처리하기 위한 토큰)
# OOV 토큰은 훈련 데이터에 없는 단어를 처리하기 위해 사용
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')

# fit_on_texts() : 텍스트 데이터에 맞게 토크나이저를 학습
# preprocessed_sentences : 전처리된 문장 리스트
print(f'preprocessed_sentences: {preprocessed_sentences}')
tokenizer.fit_on_texts(preprocessed_sentences)

tokenizer.word_index  # 단어-인덱스 매핑 출력(corpus의 모든 단어를 대상으로 생성)


preprocessed_sentences: [['little', 'prince', 'written', 'antoine', 'poetic', 'tale', 'young', 'prince', 'travels', 'home', 'planet', 'earth'], ['story', 'begins', 'pilot', 'stranded', 'sahara', 'desert', 'plane', 'crashes'], ['trying', 'fix', 'plane', 'meets', 'mysterious', 'young', 'boy', 'little', 'prince'], ['little', 'prince', 'comes', 'small', 'asteroid', 'called', 'lives', 'alone', 'rose', 'loves', 'deeply'], ['recounts', 'journey', 'pilot', 'describing', 'visits', 'several', 'planets'], ['planet', 'inhabited', 'different', 'character', 'king', 'vain', 'man', 'drunkard', 'businessman', 'geographer', 'fox'], ['encounters', 'prince', 'learns', 'valuable', 'lessons', 'love', 'responsibility', 'nature', 'adult', 'behavior'], ['earth', 'little', 'prince', 'meets', 'various', 'creatures', 'including', 'fox', 'teaches', 'relationships', 'importance', 'taming', 'means', 'building', 'ties', 'others'], ['fox', 'famous', 'line', 'become', 'responsible', 'forever', 'tamed', 'resonates', 'pr

{'<OOV>': 1,
 'prince': 2,
 'little': 3,
 'pilot': 4,
 'rose': 5,
 'fox': 6,
 'young': 7,
 'planet': 8,
 'earth': 9,
 'story': 10,
 'plane': 11,
 'meets': 12,
 'asteroid': 13,
 'lessons': 14,
 'love': 15,
 'importance': 16,
 'written': 17,
 'antoine': 18,
 'poetic': 19,
 'tale': 20,
 'travels': 21,
 'home': 22,
 'begins': 23,
 'stranded': 24,
 'sahara': 25,
 'desert': 26,
 'crashes': 27,
 'trying': 28,
 'fix': 29,
 'mysterious': 30,
 'boy': 31,
 'comes': 32,
 'small': 33,
 'called': 34,
 'lives': 35,
 'alone': 36,
 'loves': 37,
 'deeply': 38,
 'recounts': 39,
 'journey': 40,
 'describing': 41,
 'visits': 42,
 'several': 43,
 'planets': 44,
 'inhabited': 45,
 'different': 46,
 'character': 47,
 'king': 48,
 'vain': 49,
 'man': 50,
 'drunkard': 51,
 'businessman': 52,
 'geographer': 53,
 'encounters': 54,
 'learns': 55,
 'valuable': 56,
 'responsibility': 57,
 'nature': 58,
 'adult': 59,
 'behavior': 60,
 'various': 61,
 'creatures': 62,
 'including': 63,
 'teaches': 64,
 'relationships'

In [ ]:
tokenizer.index_word  # 인덱스-단어 매핑 출력

{1: '<OOV>',
 2: 'prince',
 3: 'little',
 4: 'pilot',
 5: 'rose',
 6: 'fox',
 7: 'young',
 8: 'planet',
 9: 'earth',
 10: 'story',
 11: 'plane',
 12: 'meets',
 13: 'asteroid',
 14: 'lessons',
 15: 'love',
 16: 'importance',
 17: 'written',
 18: 'antoine',
 19: 'poetic',
 20: 'tale',
 21: 'travels',
 22: 'home',
 23: 'begins',
 24: 'stranded',
 25: 'sahara',
 26: 'desert',
 27: 'crashes',
 28: 'trying',
 29: 'fix',
 30: 'mysterious',
 31: 'boy',
 32: 'comes',
 33: 'small',
 34: 'called',
 35: 'lives',
 36: 'alone',
 37: 'loves',
 38: 'deeply',
 39: 'recounts',
 40: 'journey',
 41: 'describing',
 42: 'visits',
 43: 'several',
 44: 'planets',
 45: 'inhabited',
 46: 'different',
 47: 'character',
 48: 'king',
 49: 'vain',
 50: 'man',
 51: 'drunkard',
 52: 'businessman',
 53: 'geographer',
 54: 'encounters',
 55: 'learns',
 56: 'valuable',
 57: 'responsibility',
 58: 'nature',
 59: 'adult',
 60: 'behavior',
 61: 'various',
 62: 'creatures',
 63: 'including',
 64: 'teaches',
 65: 'relationsh

In [50]:
tokenizer.word_counts  # 단어-빈도수 매핑 출력

OrderedDict([('little', 6),
             ('prince', 9),
             ('written', 1),
             ('antoine', 1),
             ('poetic', 1),
             ('tale', 1),
             ('young', 2),
             ('travels', 1),
             ('home', 1),
             ('planet', 2),
             ('earth', 2),
             ('story', 2),
             ('begins', 1),
             ('pilot', 4),
             ('stranded', 1),
             ('sahara', 1),
             ('desert', 1),
             ('plane', 2),
             ('crashes', 1),
             ('trying', 1),
             ('fix', 1),
             ('meets', 2),
             ('mysterious', 1),
             ('boy', 1),
             ('comes', 1),
             ('small', 1),
             ('asteroid', 2),
             ('called', 1),
             ('lives', 1),
             ('alone', 1),
             ('rose', 3),
             ('loves', 1),
             ('deeply', 1),
             ('recounts', 1),
             ('journey', 1),
             ('describing', 

In [ ]:
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)  # 문장들을 정수 시퀀스로 변환
sequences
# 정수 시퀀스로 변환한다는 것은 각 단어를 해당하는 인덱스로 대체하는 것을 의미.

# 결과 : 
#    인덱스 15 미만의 인덱스로 정수 인코딩을 진행하므로, 14까지의 인덱스를 확인할 수 있다.
#    인덱스 1 = oov 토큰으로, 사전에 없는 단어는 모두 1로 인코딩된다.

[[3, 2, 1, 1, 1, 1, 7, 2, 1, 1, 8, 9],
 [10, 1, 4, 1, 1, 1, 11, 1],
 [1, 1, 11, 12, 1, 7, 1, 3, 2],
 [3, 2, 1, 1, 13, 1, 1, 1, 5, 1, 1],
 [1, 1, 4, 1, 1, 1, 1],
 [8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6],
 [1, 2, 1, 1, 14, 1, 1, 1, 1, 1],
 [9, 3, 2, 12, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1],
 [6, 1, 1, 1, 1, 1, 1, 1, 2, 1, 5],
 [1, 3, 2, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 4, 1, 1, 13, 1, 5],
 [10, 1, 4, 1, 14, 1, 3, 2, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]